In [59]:
from predictors import PeriodicMeanStd
import pandas as pd 
import numpy as np

In [60]:
model = PeriodicMeanStd(flag_week=True, flag_hour=True, flag_minute=True)

# Train

In [61]:
#Data train
path = "/home/gastong/Documentos/TELCO/v0/"
filenames = ["TELCO_data_2021_01.zip", "TELCO_data_2021_02.zip", "TELCO_data_2021_03.zip"]
data_train = pd.DataFrame()
for i in range(3):
    data_train = pd.concat([data_train, pd.read_csv(path+filenames[i])])


In [62]:
def set_index(dataRaw):
    col_time = dataRaw.columns[0]
    dataRaw[col_time] = pd.to_datetime(dataRaw[col_time])
    dataRaw = dataRaw.set_index(col_time)
    return dataRaw

def fill(dataRaw):
    X = dataRaw.copy()
    X.fillna(method='ffill', inplace=True) 
    X.fillna(method='bfill', inplace=True)
    return X

def outliers(dataRaw, max_std):
    X = dataRaw.copy()
    
    if outliers:
        z_scores = (X-X.mean())/X.std()
        abs_z_scores = np.abs(z_scores)
        filtered_entries = (abs_z_scores > max_std)
        X = X.mask(filtered_entries)
    return X

In [63]:
data_train = outliers(fill(set_index(data_train)), 5)

In [64]:
model.fit(data_train)

# Operation Point

In [65]:
model.flag_day == True

False

In [66]:
#Data val
filenames = "TELCO_data_2021_04.zip"
labels_filenames = "TELCO_labels_2021_04.zip"

data_val = pd.read_csv(path+filenames)
labels_val = pd.read_csv(path+labels_filenames)

In [67]:
model.alpha_selection(data_val, labels_val, custom_metrics=True, max_alpha=5)

IndexError: list index out of range

In [ ]:
len(model.conditions)

1